In [12]:
import pandas as pd

df = pd.read_csv("data/balanced_output.csv")
print(df.head())

                                                Text  Label         ID
0  ADVERTISEMENT\n\nIt's not just Osama bin Laden...  Human  D1-178627
1  Namaste Solar Inc.’s Development Strategy Case...  Human   D1-62612
2  Revolutionary Movements and Wars of Rebellion ...  Human   D1-47706
3  Harnett County Public Health IT Department Ess...  Human   D1-68331
4  More innovative gear from Bastl Instruments.\n...  Human  D1-150166


In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["Text"])  
sequences = tokenizer.texts_to_sequences(df["Text"])
padded_sequences = pad_sequences(sequences, padding="post")

df["Label"] = df["Label"].map({"Human": 0, "AI": 1})  # Convertendo para números

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df["Label"], test_size=0.3, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout

model_dnn = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=X_train.shape[1]),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid")
])

model_dnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model_dnn.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=32)


/home/luis/miniconda3/envs/ap/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100


2025-03-18 20:52:14.204819: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (mklcpu) ran out of memory trying to allocate 2.07GiB (rounded to 2225471488)requested by op sequential_3/dense_9/kernel/Initializer/stateless_random_uniform
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-03-18 20:52:14.205402: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for mklcpu
2025-03-18 20:52:14.205416: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-03-18 20:52:14.205417: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in 

In [ ]:
from tensorflow.keras.layers import SimpleRNN

model_rnn = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=X_train.shape[1]),
    SimpleRNN(64, return_sequences=True),
    SimpleRNN(32),
    Dense(1, activation="sigmoid")
])

model_rnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model_rnn.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=32)


In [ ]:
from tensorflow.keras.layers import LSTM

model_lstm = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=X_train.shape[1]),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(1, activation="sigmoid")
])

model_lstm.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model_lstm.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)


In [ ]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_bert = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Pré-processar o texto
def encode_texts(texts):
    return tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="tf")

train_encodings = encode_texts(df["text"].tolist())
train_labels = df["label"].values

# Treinar o modelo
model_bert.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_bert.fit(train_encodings["input_ids"], train_labels, epochs=3, batch_size=8)


In [ ]:
from sklearn.metrics import classification_report

y_pred_dnn = (model_dnn.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred_dnn))

y_pred_rnn = (model_rnn.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred_rnn))

y_pred_lstm = (model_lstm.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred_lstm))